## Importing the libraries:


In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
import re
import xlsxwriter

## Importing the datasets:


In [4]:
# Importing dataset of balance and P&L function:

balance2025 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025.xlsx')
balance2024 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2024.xlsx')
balance2023 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2023.xlsx')
balance2022 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2022.xlsx')
balance2021 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2021.xlsx')
balance2020 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2020.xlsx')

pnl2025 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2025.xlsx')
pnl2024 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2024.xlsx')
pnl2023 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2023.xlsx')
pnl2022 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2022.xlsx')
pnl2021 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2021.xlsx')
pnl2020 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2020.xlsx')

# Saving in lists for functions:
balance_list = [balance2025, balance2024, balance2023, balance2022, balance2021, balance2020]
pnl_list = [pnl2025, pnl2024, pnl2023, pnl2022, pnl2021, pnl2020]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025.xlsx'

## Cleaning the data with functions:


In [ ]:
# Unnecessary column removal from list of dataframes:
def remove_unnecessary_columns(df_list):
    for df in df_list:
        remove_columns = ['ja_pavadinimas', 'form_pavadinimas','template_name', 'standard_name', 'line_type_id']
        for col in remove_columns:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
    return df_list
